# Packages Installs, imports, and presets

In [7]:
from scapy.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
import sklearn

import feature_engine
import feature_engine.encoding

import feature_engine.imputation
from constants import TIMEOUT, NUMBER_PACKETS_OPTIONS
import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [8]:
# df_train= pd.read_parquet('data/df_train_statistics.parquet')
# df_validation = pd.read_parquet('data/df_validation_statistics.parquet')
# df_test=pd.read_parquet('data/df_test_statistics.parquet')

In [17]:
# TIMEOUT = 30
    # df_train_statistics.to_parquet(f'data/iec104_train_statistics_{TIMEOUT}_{number_packets}.parquet')

    # df_validation_statistics = get_flows_statistics(df_validation)
    # df_validation_statistics.to_parquet(f'data/iec104_validation_statistics_{TIMEOUT}_{number_packets}.parquet')

    # df_test_statistics = get_flows_statistics(df_test)
    # df_test_statistics.to_parquet(f'data/iec104_train_statistics_{TIMEOUT}_{number_packets}.parquet')
for number_packets in NUMBER_PACKETS_OPTIONS:
    df_train=pd.read_parquet(f'data/iec104_train_statistics_{TIMEOUT}_{number_packets}.parquet')

    df_validation=pd.read_parquet(f'data/iec104_validation_statistics_{TIMEOUT}_{number_packets}.parquet')

    df_test=pd.read_parquet(f'data/iec104_train_statistics_{TIMEOUT}_{number_packets}.parquet')

    # df_train

    df_test=df_test.drop('flow',axis=1)
    df_validation=df_validation.drop('flow',axis=1)
    df_train=df_train.drop('flow',axis=1)

    df_train.columns
    tmp = df_train.dtypes
    tmp.loc[tmp=='object']
    oe = feature_engine.encoding.OrdinalEncoder(encoding_method='arbitrary')
    df_train[['src','dst']]
    oe.fit(df_train[['src','dst']])
    df_train[['src','dst']]=oe.transform(df_train[['src','dst']])
    df_validation[['src','dst']]=oe.transform(df_validation[['src','dst']])
    df_test[['src','dst']]=oe.transform(df_test[['src','dst']])
    oe = feature_engine.encoding.OrdinalEncoder()
    tmp = df_train.isnull().sum()
    tmp.loc[tmp>0]
    columns_with_nan_values = list(tmp.loc[tmp>0].index)
    
    mmi = feature_engine.imputation.ArbitraryNumberImputer(arbitrary_number=0)
    mmi.fit(df_train[columns_with_nan_values])

    df_train[columns_with_nan_values]=mmi.transform(df_train[columns_with_nan_values])
    df_test[columns_with_nan_values]=mmi.transform(df_test[columns_with_nan_values])
    df_validation[columns_with_nan_values]=mmi.transform(df_validation[columns_with_nan_values])
    
    X_train = df_train.loc[:,df_train.columns != 'label']
    y_train = df_train.loc[:,df_train.columns == 'label'].to_numpy().flatten()
    X_test = df_test.loc[:,df_test.columns != 'label']
    y_test = df_test.loc[:,df_test.columns == 'label'].to_numpy().flatten()
    X_val = df_validation.loc[:,df_validation.columns != 'label']
    y_val = df_validation.loc[:,df_validation.columns == 'label'].to_numpy().flatten()

    models = {
        "Logistic Regression": (LogisticRegression(max_iter=1000), {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear']
        }),
        "Linear Discriminant Analysis": (LinearDiscriminantAnalysis(), {
            'solver': ['svd', 'lsqr', 'eigen']
        }),
        "Quadratic Discriminant Analysis": (QuadraticDiscriminantAnalysis(), {
            'reg_param': [0.0, 0.1, 0.5, 1.0]
        }),

        "Naive Bayes": (GaussianNB(), {
            'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
        }),
        "SVM": (SVC(probability=True), {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
        }),
        "Random Forest": (RandomForestClassifier(), {
            'n_estimators': [50, 100, 200],
            'max_features': ['sqrt'],
            'max_depth': [None, 10, 20, 30]
        }),
        "Adaboost": (AdaBoostClassifier(algorithm='SAMME'), {
            'n_estimators': [50, 200],
            'learning_rate': [0.01, 0.1]
        }),
        "MLP": (MLPClassifier(max_iter=1000), {
        'hidden_layer_sizes': [ (100,50,50)],
        'activation': ['tanh','relu'],
        'solver': [ 'lbfgs','adam'],
        'alpha': [0.0001],
        'learning_rate': [ 'adaptive']
        }),
        "Decision Tree": (DecisionTreeClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30]
        }),
    }

    # Train and evaluate models using HalvingGridSearchCV
    results = []
    NUM_RUNS = 20
    for name, (model, param_grid) in models.items():
        grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=16,verbose=3)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        # print(best_model)
        # best_model.set_params({'max_iter': 10000})
        best_model.fit(X_train, y_train)
        
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        prediction_times = []
        for _ in tqdm(range(NUM_RUNS)):
            # print(X_test)
            # for c, sample in X_test.iterrows():
            start_time = time.time()
            # print(sample)
            best_model.predict(X_test)
            end_time = time.time()
            duration = (end_time - start_time)/len(X_test)
            prediction_times.append([duration])
            # prediction_times.append( / len(X_test))
        prediction_times=pd.DataFrame(prediction_times)
        
        # Generate the confusion matrix
        cm = confusion_matrix(y_test, y_pred)
            
        results.append({
        "Model": name,
        "Best Parameters": grid_search.best_params_,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "Confusion Matrix": cm,
        "Average Prediction Time (s/sample)": prediction_times.mean().values[0],
        "Standard Deviation Prediction Time (s/sample)": prediction_times.std().values[0],
        "Average Throughput (samples/s)": 1 / prediction_times.mean().values[0],
        "Standard Deviation Throughput (samples/s)": 1 / prediction_times.std().values[0],
        "Average Total Prediction Time (s/sample)": prediction_times.iloc[:, 0].mean() + X_test['computation_time'].mean(),
        "Standard Deviation Total Prediction Time (s/sample)": prediction_times.iloc[:, 0].std() + X_test['computation_time'].std(),
        "Average Total Throughput (samples/s)": 1 / (prediction_times.iloc[:, 0].mean() + X_test['computation_time'].mean()),
        "Standard Deviation Total Throughput (samples/s)": 1 / (prediction_times.iloc[:, 0].std() + X_test['computation_time'].std())
    })

    # Convert results to a DataFrame
    df_results = pd.DataFrame(results)
    df_results['output_date'] = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
    df_results['number_packets'] = number_packets
    df_results['timeout'] = TIMEOUT
    output_path= 'data/results.csv'
    df_results.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    # Print the results
    # print(df_results)

    # # Plot confusion matrices for each model
    # for result in results:
    #     cm = result["Confusion Matrix"]
    #     model_name = result["Model"]
        
    #     plt.figure(figsize=(8, 6))
    #     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    #     plt.title(f'Confusion Matrix for {model_name}')
    #     plt.xlabel('Predicted')
    #     plt.ylabel('True')
    #     plt.show()

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 248.27it/s]


Fitting 3 folds for each of 3 candidates, totalling 9 fits


/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
3 fits failed out of a total of 9.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/discriminant_an

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 24.51it/s]


Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/hwerneck/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 34.45it/s]


Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 